# **Importando Bibliotecas**

In [1]:
import re
import pandas as pd
from io import StringIO

# **Extraindo Dados**

In [2]:
with open("Dados/Landing/1410035501-noSymbol.csv", "r") as file:
    conteudo = file.read()

    texto_csv = ""

    conteudo_linhas = conteudo.split("\n")
    conteudo_linhas = conteudo_linhas[11:32]
    del conteudo_linhas[1]

    colunas = conteudo_linhas[0]

    lista_meses = colunas.split(",")
    meses = len(lista_meses[1:])

    for i, linha in enumerate(conteudo_linhas):

        valores = linha.split('"')

        valores = valores[1::2]

        valor_final = ";".join(valores)

        texto_csv += f"{valor_final}\n"


# **Transformando Dados**

In [ ]:
arquivo_csv = StringIO(texto_csv)

df = pd.read_csv(arquivo_csv, sep=";")

display(df)

In [ ]:
print(df.columns)

In [5]:
#Renomeando a primeira coluna
df = df.rename(columns={"North American Industry Classification System (NAICS) 6": "North American Industry Classification System (NAICS)"})

In [ ]:
first = df.columns[0]

df[first] = df[first].apply(lambda x: re.sub(r"\d", "", x))

display(df[first])

In [ ]:
display(df.head())

In [ ]:


df.iloc[:,1:] = df.iloc[:,1:].map(lambda cel: re.sub(",","", cel))
df.iloc[:,1:] = df.iloc[:,1:].astype(float)



display(df)

In [ ]:
# Traduzindo as colunas
column_translation_dict = {
    'North American Industry Classification System (NAICS)': 'Sistema de Classificação de Indústrias da América do Norte (SCIAN)',
    'January': 'Janeiro',
    'February': 'Fevereiro',
    'March': 'Março',
    'April': 'Abril',
    'May': 'Maio',
    'June': 'Junho',
    'July': 'Julho',
    'August': 'Agosto',
    'September': 'Setembro',
    'October': 'Outubro',
    'November': 'Novembro',
    'December': 'Dezembro'
}


def traduzir_coluna(column):
    for eng, ptbr in column_translation_dict.items():
        if eng in column:
            column = column.replace(eng, ptbr)
    return column


df.columns = df.columns.map(lambda column: column.strip())
df.columns = df.columns.map(traduzir_coluna)

display(df)

In [ ]:
# Traduzindo a primeira coluna
first = df.columns[0]

cell_translation_dict = {
    "Total employed, all industries": "Total empregado, todas as indústrias",
    "Goods-producing sector": "Setor de produção de bens",
    "Agriculture": "Agricultura",
    "Forestry, fishing, mining, quarrying, oil and gas": "Silvicultura, pesca, mineração, extração de pedreiras, petróleo e gás",
    "Utilities": "Serviços públicos",
    "Construction": "Construção",
    "Manufacturing": "Manufatura",
    "Services-producing sector": "Setor de produção de serviços",
    "Wholesale and retail trade": "Comércio atacadista e varejista",
    "Transportation and warehousing": "Transporte e armazenamento",
    "Finance, insurance, real estate, rental and leasing": "Finanças, seguros, imóveis, aluguel e leasing",
    "Professional, scientific and technical services": "Serviços profissionais, científicos e técnicos",
    "Business, building and other support services": "Serviços de negócios, construção e outros serviços de apoio",
    "Educational services": "Serviços educacionais",
    "Health care and social assistance": "Cuidados com a saúde e assistência social",
    "Information, culture and recreation": "Informação, cultura e recreação",
    "Accommodation and food services": "Acomodações e serviços de alimentação",
    "Other services (except public administration)": "Outros serviços (exceto administração pública)",
    "Public administration": "Administração pública"
}


def traduzir_cell(cell):
    for eng, ptbr in cell_translation_dict.items():
        if eng in cell:
            cell = cell.replace(eng,ptbr)
    return cell


df[first] = df[first].map(lambda cell: cell.strip())
df[first] = df[first].map(traduzir_cell)

display(df)

In [11]:
df.to_csv("Dados/Tratados/DadosSetoresCanada.csv", sep=";", index=False)

# **Loading na S3**

In [12]:
import os
import boto3
from dotenv import load_dotenv


# Carregar variáveis
load_dotenv(r"credentials/.env")

# Configure as cresdencias da AWS
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

# Crie um client S3
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

# Carregue o arquivo para o S3
s3.upload_file(
    Filename='Dados/Tratados/DadosSetoresCanada.csv',
    Bucket='projetofinal-consuelo',
    Key='DadosSetoresCanada.csv'
)